In [168]:
import math
import pandas as pd
import numpy as np
import feather
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import metrics
from sklearn import ensemble
import matplotlib.pyplot as plt
%matplotlib inline

Importing trip data for March 2016 and Weather data from Weather Underground. 

The source data is read in a separate Python class. It is manipulated as a DataFrame and finally exported into Feather format. The exported file is imported in this notebook. To read more about Feather, read https://blog.rstudio.org/2016/03/29/feather/

The benefit of doing this were:
- The ETL part of the analysis is in a separate file (CitiBike_ETL.py)
- The Feather file can read in R also
- During analysis, this notebook will be run multiple times and it is efficient to just import the DataFrame

In [169]:
bikedata = feather.read_dataframe('../../Data/CitiBike_Data/bikedata.feather')
print bikedata.columns
print len(bikedata)

Index([u'bikeid', u'birth year', u'dtstartdatehour', u'dtstopdatehour',
       u'end station id', u'end station latitude', u'end station longitude',
       u'end station name', u'female', u'male', u'start station id',
       u'start station latitude', u'start station longitude',
       u'start station name', u'startdatehour', u'starttime', u'stoptime',
       u'tripduration', u'usertype'],
      dtype='object')
10454235


In [170]:
bikedata.groupby(by='start station name').bikeid.count().sort_values(ascending=False).head(10)

start station name
Pershing Square North    120672
Lafayette St & E 8 St    104621
8 Ave & W 31 St          104413
W 21 St & 6 Ave           99316
E 17 St & Broadway        91051
Broadway & E 14 St        78803
Broadway & E 22 St        74463
Greenwich Ave & 8 Ave     74062
8 Ave & W 33 St           73087
W 41 St & 8 Ave           71850
Name: bikeid, dtype: int64

In [178]:
# How many bikes will leave a particular station at a particular time?
# First let's start with a single station. In our case, let's start with E 40 St & 5 Ave
# dfFocusStation = bikedata[(bikedata['start station name'] == "E 40 St & 5 Ave") & (bikedata['usertype'] == 'Subscriber')]
dfFocusStation = bikedata[(bikedata['start station name'] == "Pershing Square North")]

dfGroupBy = dfFocusStation.groupby(by=['dtstartdatehour'])

# Number of departures per hour
departures = dfGroupBy.bikeid.count().reset_index()

# Date column
departures['startdatehour'] = dfGroupBy.startdatehour.max().reset_index().startdatehour
departures['startdate'] = departures.dtstartdatehour.dt.date.apply(lambda x:x.strftime('%Y-%m-%d'))

# Male to female ratio
sum_male = dfGroupBy.male.sum().reset_index().male
sum_female = dfGroupBy.female.sum().reset_index().female
departures['male_to_female_ratio'] = (sum_male).astype(float) / (sum_male + sum_female)

print departures.head()

      dtstartdatehour  bikeid        startdatehour   startdate  \
0 2015-01-01 03:00:00       1  2015-01-01 03:00:00  2015-01-01   
1 2015-01-01 12:00:00       1  2015-01-01 12:00:00  2015-01-01   
2 2015-01-01 14:00:00       2  2015-01-01 14:00:00  2015-01-01   
3 2015-01-01 16:00:00       2  2015-01-01 16:00:00  2015-01-01   
4 2015-01-01 17:00:00       2  2015-01-01 17:00:00  2015-01-01   

   male_to_female_ratio  
0                   1.0  
1                   1.0  
2                   1.0  
3                   0.5  
4                   1.0  


In [179]:
# Import holiday data
holiday = pd.read_csv('./data/public_holiday.csv')

# Merge with departures dataframe
holiday_df = pd.merge(departures, holiday, left_on='startdate', right_on = 'Date', how='left')
holiday_df.Holiday.fillna(0, inplace=True)
holiday_df['dayofweek'] = holiday_df['dtstartdatehour'].dt.dayofweek.values

holiday_df.loc[holiday_df.dayofweek <= 4, 'day_type'] = 0 #'weekday'
holiday_df.loc[holiday_df.dayofweek > 4, 'day_type'] = 1 #'weekend'
holiday_df.loc[holiday_df.Holiday <> 0, 'day_type'] = 2 #'holiday'

holiday_df.drop('Date', axis=1, inplace=True)
holiday_df.drop('Holiday', axis=1, inplace=True)
holiday_df.drop('dayofweek', axis=1, inplace=True)
holiday_df.drop('dtstartdatehour', axis=1, inplace=True)
holiday_df.drop('startdate', axis=1, inplace=True)

holiday_df[holiday_df.day_type == 2].head()

,bikeid,startdatehour,male_to_female_ratio,day_type
0,1,2015-01-01 03:00:00,1.0,2
1,1,2015-01-01 12:00:00,1.0,2
2,2,2015-01-01 14:00:00,1.0,2
3,2,2015-01-01 16:00:00,0.5,2
4,2,2015-01-01 17:00:00,1.0,2


In [185]:
# Join with weather data
weather_file = './data/temperature/weather.csv'
weather = pd.read_csv(weather_file)
weather.drop('Unnamed: 0', axis=1, inplace=True)


In [186]:
# final_df = pd.merge(departures, weather, on='date', how='left')
final_df = pd.merge(holiday_df, weather, left_on='startdatehour', right_on = 'date', how='left')
final_df.head()

,bikeid,startdatehour,male_to_female_ratio,day_type,date,temp,fog,rain,snow,hail,thunder,tornado,visi,dewptm,humidity,wind_speed
0,1,2015-01-01 03:00:00,1.0,2,2015-01-01 03:00:00,-2.8,0,0,0,0,0,0,10,-13.9,43,5.6
1,1,2015-01-01 12:00:00,1.0,2,2015-01-01 12:00:00,1.7,0,0,0,0,0,0,10,-13.9,31,16.7
2,2,2015-01-01 14:00:00,1.0,2,2015-01-01 14:00:00,3.3,0,0,0,0,0,0,10,-13.3,29,14.8
3,2,2015-01-01 16:00:00,0.5,2,2015-01-01 16:00:00,3.9,0,0,0,0,0,0,10,-11.7,32,18.5
4,2,2015-01-01 17:00:00,1.0,2,2015-01-01 17:00:00,3.9,0,0,0,0,0,0,10,-10.6,34,11.1


In [187]:
final_df.describe()
# plt.bar(final_df.date, final_df.bikeid)
# final_df.isna()

,bikeid,male_to_female_ratio,day_type,temp,fog,rain,snow,hail,thunder,tornado,visi,dewptm,humidity,wind_speed
count,8360.000000,8360.000000,8360.000000,8360.000000,8360.000000,8360.000000,8360.000000,8360,8360,8360,8360.000000,8360.000000,8360.000000,8360.000000
mean,14.434450,0.868767,0.311603,12.624246,0.002392,0.067943,0.028349,0,0,0,-328.843828,3.935012,58.547847,8.013995
std,20.332292,0.190436,0.532166,10.176818,0.048856,0.251662,0.165979,0,0,0,1806.893222,11.124053,19.090252,6.013627
min,1.000000,0.000000,0.000000,-18.300000,0.000000,0.000000,0.000000,0,0,0,-9999.000000,-28.300000,11.000000,0.000000
25%,3.000000,0.818182,0.000000,4.400000,0.000000,0.000000,0.000000,0,0,0,9.000000,-3.900000,43.000000,5.600000
50%,7.000000,0.923077,0.000000,13.000000,0.000000,0.000000,0.000000,0,0,0,10.000000,5.000000,57.000000,7.400000
75%,16.000000,1.000000,1.000000,21.100000,0.000000,0.000000,0.000000,0,0,0,10.000000,13.300000,75.000000,11.100000
max,154.000000,1.000000,2.000000,35.600000,1.000000,1.000000,1.000000,0,0,0,10.000000,23.300000,100.000000,37.000000


In [198]:
# # LabelEncoder for the dtstartdatehour column
# le = preprocessing.LabelEncoder()
# le.fit(final_df.dtstartdatehour)
# final_df['startdatehour'] = le.transform(final_df.dtstartdatehour)
# # LabelEncoder ends

cols = [col for col in final_df.columns if col not in ['bikeid', 'startdatehour', 'date']]

x = final_df[cols]
# y = final_df.bikeid
y = final_df.bikeid.apply(lambda x: math.log(x+1))

X_train, X_test, y_train, y_test = cross_validation.train_test_split(x, y, random_state = 1)
print cols

['male_to_female_ratio', 'day_type', 'temp', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'visi', 'dewptm', 'humidity', 'wind_speed']


In [199]:
###
# Linear regression
###
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
rmse = metrics.mean_squared_error(y_test, pred)
r2_score = metrics.r2_score(y_test, pred)

print 'MSE: ', rmse
print 'R2 score: ', r2_score

MSE:  0.903065880439
R2 score:  0.166941843794


In [200]:
###
# Ridge regression
###
ridge = linear_model.RidgeCV(alphas=[0.0001, 0.001, 0.1, 1.0, 10.0, 100.0, 1000.0], 
                             normalize=True, scoring='mean_squared_error')
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
print 'MSE (RidgeCV): ', metrics.mean_squared_error(y_test, ridge_pred)
ridge_r2_score = metrics.r2_score(y_test, ridge_pred)
print 'R2 Score (RidgeCV): ', ridge_r2_score

MSE (RidgeCV):  0.903050798594
R2 Score (RidgeCV):  0.16695575646


In [201]:
###
# Lasso regression
###
lasso = linear_model.Lasso(alpha = 0.00009, normalize=True)
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
print 'MSE (Lasso): ', metrics.mean_squared_error(y_test, lasso_pred)
lasso_r2_score = metrics.r2_score(y_test, lasso_pred)
print 'R2 Score (Lasso): ', lasso_r2_score
for i, item in enumerate(lasso.coef_):
    print cols[i], item

MSE (Lasso):  0.903945033054
R2 Score (Lasso):  0.166130845092
male_to_female_ratio 0.367341395203
day_type -0.586771835207
temp 0.0253637747648
fog 0.179824580065
rain -0.00147431039377
snow -0.0250568599688
hail 0.0
thunder 0.0
tornado 0.0
visi -2.32991525372e-05
dewptm 0.0
humidity -0.00195945263378
wind_speed 0.00156330163937


In [202]:
# Based on the results above, we find that some of the coefficients are 0 and some are negative.
# We'll keep only the positive ones

rf_cols = [col for col in final_df.columns if col not in ['bikeid', 'startdatehour', 'date', 'visi', 'snow',
                                                          'hail', 'thunder', 'tornado', 'humidity', 'dewptm', 'wind_speed']]

rf_x = final_df[rf_cols]
rf_X_train, rf_X_test, rf_y_train, rf_y_test = cross_validation.train_test_split(rf_x, y, random_state = 1)

In [203]:
###
# Random Forest regression
###
rfr = ensemble.RandomForestRegressor(n_estimators=50, bootstrap=True)
rfr.fit(rf_X_train, rf_y_train)
rf_pred = rfr.predict(rf_X_test)
print 'MSE (Random Forest): ', metrics.mean_squared_error(rf_y_test, rf_pred)
rf_r2_score = metrics.r2_score(rf_y_test, rf_pred)
print 'R2 Score (Random Forest): ', rf_r2_score

MSE (Random Forest):  0.338526415615
R2 Score (Random Forest):  0.687716923285
